Grupo 5:
Flor, Cristian, Gabriel

In [364]:
# 1 Inicio
# 1.1 Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 1.2 Abrir los archivos CSV
df_cash = pd.read_csv('extract - cash request - data analyst.csv')
df_fees = pd.read_csv('extract - fees - data analyst - .csv')

# 1.3 Mostrar las primeras filas de los DataFrames
print("\n\033[1m\033[95mPARTE 1: Primeras filas\033[0m")
print("Cash head:")
display(df_cash.head())
print("Fees head:")
display(df_fees.head())

In [ ]:
# 2. Información de los DataFrames
# 2.1 Mostrar información de los tipos de datos
print("\n\033[1m\033[95mPARTE 2: Descripción estadística\033[0m")
print("Info df_cash:")
print(df_cash.info())
print("----------------------------------------------------------")
print("Info df_fees:")
print(df_fees.info())
print("----------------------------------------------------------")
# 2.2 Mostrar descripción estadística
#print("Descripción estadística de Cash:")
#print(df_cash.describe())
#print("----------------------------------------------------------")
#print("Descripción estadística de Fees:")
#print(df_fees.describe())
#print("----------------------------------------------------------")

In [345]:
# 3. Limpiar fechas para agrupar por meses
# 3.1 Definir las columnas de fechas para Cash y Fees
#print("\n\033[1m\033[95mPARTE 3: Agrupar por meses\033[0m")
date_columns_cash = [
    'created_at', 'updated_at', 'moderated_at', 
    'reimbursement_date', 'cash_request_received_date', 
    'money_back_date', 'send_at', 'reco_creation', 
    'reco_last_update'
]
date_columns_fees = ['created_at', 'updated_at', 'paid_at', 'from_date', 'to_date']

# 3.2 Convertir y normalizar las fechas con pd.to_datetime
for col in date_columns_cash:
    df_cash[col] = pd.to_datetime(df_cash[col], errors='coerce').dt.normalize()
for col in date_columns_fees:
    df_fees[col] = pd.to_datetime(df_fees[col], errors='coerce').dt.normalize()

In [ ]:
# 4. Tipos de datos las columnas de fecha de ambos df:
print("\n\033[1m\033[95mPARTE 4: Tipos de datos las columnas de fecha de ambos df:\033[0m")
print("\nTipos de datos de Cash Requests:")
print(df_cash[date_columns_cash].dtypes)
print("\nTipos de datos de Fees:")
print(df_fees[date_columns_fees].dtypes)

In [ ]:
# 5.1 Ordenar por 'created_at' en ambos DataFrames
df_cash_sorted = df_cash.sort_values(by='created_at', ascending=True)
df_fees_sorted = df_fees.sort_values(by='created_at', ascending=True)

# 5.2 Mostrar los primeros resultados después de ordenar
print("\n\033[1m\033[95mPARTE 5: Resultados después de ordenar por fecha de creación:\033[0m")
print("Cash sorted by created_at:")
display(df_cash_sorted.head())
print("Fees sorted by created_at:")
display(df_fees_sorted.head())

In [ ]:
# 6 Creacion de cohortes
print("\n\033[1m\033[95mPARTE 6: Creación de cohortes\033[0m")
# 6.1 Llenar valores nulos de 'user_id' con 'deleted_account_id'
df_cash['user_id'] = df_cash['user_id'].fillna(df_cash['deleted_account_id'])

# 6.2 Eliminar filas con user_id aún nulos
df_cash = df_cash[~df_cash['user_id'].isna()]

# 6.3 Agrupar por user_id y obtener la primera fecha de created_at
cohorts = df_cash.groupby('user_id')['created_at'].min().reset_index()
cohorts.rename(columns={'created_at': 'first_created_at'}, inplace=True)

# 6.4 Crear una nueva columna con el formato de mes y año
cohorts['cohort'] = cohorts['first_created_at'].dt.tz_localize(None).dt.to_period('M').astype(str)

# 6.5 Unir los cohorts al DataFrame original
df_cash = df_cash.merge(cohorts[['user_id', 'cohort']], on='user_id', how='left')

# 6.6 Mostrar los primeros resultados para verificar
display(df_cash[['user_id', 'created_at', 'cohort']].head())

In [ ]:
# 7. Ordenar el DataFrame df_cash por la columna created_at
df_cash_sorted = df_cash.sort_values(by='created_at', ascending=True)

# 7.1 Mostrar los primeros resultados después de ordenar
print("\n\033[1m\033[95mPARTE 7: Ordenar\033[0m")
print("DataFrame cash ordenado por created_at:")
display(df_cash_sorted[['user_id', 'created_at', 'cohort', 'deleted_account_id']])

In [ ]:
# 8. Gráfico - Distribución de usuarios por cohorts
print("\n\033[1m\033[95mPARTE 8: Distribución de usuarios por cohorte\033[0m")

# Contar usuarios únicos por cohort
cohort_counts = df_cash.groupby('cohort')['user_id'].nunique()

# 8.1 Visualización de la distribución de usuarios por cohort
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=cohort_counts.index.astype(str), y=cohort_counts.values, hue=cohort_counts.index.astype(str), palette='Set2', legend=False)
plt.title('Distribución de usuarios por cohorte')
plt.xlabel('Cohorte (mes y año)')
plt.ylabel('Número de usuarios únicos')
plt.xticks(rotation=45)

# 8.2 Añadir la cantidad total arriba de cada columna
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='bottom', fontsize=12)

plt.show()

In [ ]:
# 9.1 Distribución de los estados actuales
print("\n\033[1m\033[95mPARTE 9: Distribución de estados de las solicitudes de dinero\033[0m")
status_distribution = df_cash['status'].value_counts()
print("\nDistribución de los estados de las solicitudes de dinero")
print(status_distribution)

# 9.2 Visualización de la distribución de los estados
plt.figure(figsize=(10, 6))
ax = sns.countplot(x='status', data=df_cash, palette='Set2', hue='status', legend=False)  # Asignar hue
plt.title('Distribución de los Estados de las Solicitudes de Dinero')
plt.xlabel('Estado de la Solicitud')
plt.ylabel('Número de Solicitudes')
plt.xticks(rotation=45)

# 9.3 Añadir la cantidad total arriba de cada barra
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}', 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='bottom', fontsize=12)
plt.show()

In [ ]:
# 10 Frecuencia media por cohorte
print("\n\033[1m\033[95mPARTE 10: Frecuencia media por cohorte\033[0m")
df_cash['created_at'] = pd.to_datetime(df_cash['created_at'])
df_cash['cohort'] = pd.to_datetime(df_cash['cohort'], format='%Y-%m')

# 10.1 Filtrar usuarios que han hecho más de una solicitud, excluyendo 'rejected'
user_counts = df_cash['user_id'].value_counts()
multiple_requests_users = user_counts[user_counts > 1].index
df_cash_filtered = df_cash[(df_cash['status'] != 'rejected') & (df_cash['user_id'].isin(user_counts[user_counts > 1].index))]

# 10.2 Calcular la frecuencia por usuario como la diferencia entre solicitudes
df_cash_filtered = df_cash_filtered.sort_values(by=['user_id', 'created_at'])
df_cash_filtered['days_between'] = df_cash_filtered.groupby('user_id')['created_at'].diff().dt.days.dropna()

# 10.3 Calcular el número de usuarios que solo han hecho una solicitud por cohorte
single_request_users = user_counts[user_counts == 1].index
df_cash_single_request = df_cash[df_cash['user_id'].isin(single_request_users) & (df_cash['status'] != 'rejected')]
single_request_count = df_cash_single_request.groupby('cohort')['user_id'].nunique().reset_index(name='single_request_count')
# Ordenar cronoogicamente los cohortes en el eje x
df_cash_filtered['cohort'] = pd.to_datetime(df_cash_filtered['cohort'])
df_cash_filtered = df_cash_filtered.sort_values(by='cohort')
single_request_count['cohort'] = pd.to_datetime(single_request_count['cohort'])
single_request_count = single_request_count.sort_values(by='cohort')

# 10.4 Visualizar el diagrama de cajas por cohort, añadiendo valores clave
plt.figure(figsize=(12, 8))
box_plot = sns.boxplot(x='cohort', y='days_between',hue='cohort', data=df_cash_filtered, palette='Set2')

# Calcular y añadir valores clave como cuartiles, mediana, media, máximo y mínimo
stats = df_cash_filtered.groupby('cohort')['days_between'].describe()

for i, cohort in enumerate(stats.index):
    median = stats.loc[cohort, '50%']
    q1 = stats.loc[cohort, '25%']
    q3 = stats.loc[cohort, '75%']
    mean = stats.loc[cohort, 'mean']
    minimum = stats.loc[cohort, 'min']
    maximum = stats.loc[cohort, 'max']
    
    # Desplazar los textos a la derecha
    plt.text(i - 0.0, median, f'{median:.1f}', ha='center', va='bottom', color='black', fontsize=10, fontweight='bold')
    plt.text(i + 0.0, q1, f'{q1:.1f}', ha='center', va='top', color='blue', fontsize=9)
    plt.text(i + 0.0, q3, f'{q3:.1f}', ha='center', va='bottom', color='blue', fontsize=9)
    plt.text(i + 0.4, mean, f'{mean:.1f}', ha='center', va='bottom', color='brown', fontsize=10)
    plt.text(i - 0.3, minimum, f'{minimum:.1f}', ha='center', va='top', color='green', fontsize=9)
    plt.text(i + 0.3, maximum, f'{maximum:.1f}', ha='center', va='bottom', color='purple', fontsize=9)

# 10.6 Ajustes visuales
plt.xticks(rotation=45)
plt.title('Distribución de Días Entre Solicitudes por Cohorte (usuarios con más de una solicitud)')
plt.xlabel('Cohorte (mes y año)')
plt.ylabel('Días Entre Solicitudes')

# 10.7 Agregar leyenda para los colores
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', label='Media', markerfacecolor='brown', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Mediana', markerfacecolor='black', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Q1', markerfacecolor='blue', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Q3', markerfacecolor='blue', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Minimo', markerfacecolor='green', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='Máximo', markerfacecolor='purple', markersize=10)
]
plt.legend(handles=legend_elements, loc='upper right', fontsize=9, title='Referencias de color')

# 10.8 Ajustar el layout para los textos desplazados
plt.subplots_adjust(left=0.1, bottom=0.2)
# 10.9 Mostrar el diagrama
plt.show()

print("-----------------------------------------------------------------------------------------------------------------------------------------------")
#10.9.1 Crear una nueva figura de tabla horizontal para incluir a los usuarios eliminados
plt.figure(figsize=(12, 2))
table = plt.table(cellText=[single_request_count['single_request_count'].values],
                  colLabels=single_request_count['cohort'].dt.strftime('%Y-%m'),
                  loc='center', cellLoc='center')

table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 1.5)

# Ocultar ejes ya que solo mostramos la tabla
plt.axis('off')
plt.title('Cantidad de usuarios eliminados que solo han hecho una solicitud')
#10.9.2 Mostrar la nueva tabla
plt.show()

In [ ]:
# 11. Analizar los ingresos generados por cohorte
print("\n\033[1m\033[95mPARTE 11: Analizar ingresos generados por cohorte\033[0m")

# 11.1 Hacemos un merge de ambos df 
df_combinados = df_fees.merge(df_cash[['id', 'cohort']], left_on='cash_request_id', right_on='id', how='left')

# 11.2 Verificamos la union de los data frame
display(df_combinados.head())

# 11.3 Contemplamos solo las comisiones que han sido aceptadas, ya que son las unicas que han generado ingresos registrados.
df_aprobados = df_combinados[(df_combinados['status'] == 'accepted')]

# 11.4 Agrupamos los ingresos por cohorte
ingresos_generados = df_aprobados[["total_amount", "cohort"]].groupby("cohort").sum()
print("Ingresos generados por cohorte:")
display(ingresos_generados)

# 11.5 Graficamos los ingresos generados por cohorte
plt.figure(figsize=(10, 6))
graf_ing = sns.barplot(x='cohort', y='total_amount', hue='cohort', data=ingresos_generados, palette='Set2', legend=False)
plt.title('Ingresos generados por cohorte')
plt.xlabel('Cohorte (mes y año)')
plt.ylabel('Ingresos totales')
plt.xticks(rotation=45)
for p in graf_ing.patches:
    graf_ing.annotate(f'{int(p.get_height())}', 
                      (p.get_x() + p.get_width() / 2., p.get_height()),  
                      ha='center', va='bottom', fontsize=12)  
plt.show()

display(df_aprobados)

In [ ]:
# 12. Distribución de estado por cohorte
print("\n\033[1m\033[95mPARTE 12: Distribución de estado por cohorte\033[0m")
cohort_status_counts = df_cash.pivot_table(index='cohort', columns='status', aggfunc='size', fill_value=0)

# Usar paleta Set2 de Seaborn
palette = sns.color_palette("Set2")

# Graficar
cohort_status_counts.plot(kind='bar', stacked=True, figsize=(10, 6), color=palette)
plt.title('Distribución de estados por cohorte')
plt.xlabel('Cohorte')
plt.ylabel('Número de registros')
plt.xticks(rotation=45)
plt.legend(title='Estado')
plt.show()

In [ ]:
# 13. Tasas de incidentes por cohorte en %
print("\n\033[1m\033[95mPARTE 13: Tasas de incidentes por cohorte en %\033[0m")

# 13.1 Agregamos una fila con los totales para cada cohorte
cohort_status_counts.loc['Total'] = cohort_status_counts.sum()
# Mostramos el DataFrame con los totales
print("Distribución de los estados de las solicitudes:")
display(cohort_status_counts)

# Definir los estados de incidentes a filtrar
incident_status = ['rejected', 'failed', 'canceled', 'error', 'direct_debit_rejected', 'direct_debit_sent', 'transaction_declined']

# 13.2 Filtrar solo los incidentes de pago
incident_df = df_cash[df_cash['status'].isin(incident_status)]

# 13.3 Contar el número de incidentes por cohorte
incident_counts = incident_df.groupby('cohort')['status'].count().reset_index(name='incidents_count')

# 13.4 Obtener el total de solicitudes por cohorte
total_requests = df_cash.groupby('cohort')['status'].count().reset_index(name='total_count')

# 13.5 Combinar ambas tablas para calcular la tasa de incidentes
cohort_incident_rate = incident_counts.merge(total_requests, on='cohort')
cohort_incident_rate['incident_rate'] = (cohort_incident_rate['incidents_count'] / cohort_incident_rate['total_count']) * 100

# 13.6 Mostrar la tasa de incidentes por cohorte
print("Tasa de incidentes por cohorte (%):")
display(cohort_incident_rate[['cohort', 'incident_rate']])

# 13.7 Visualización de la tasa de incidentes por cohorte
plt.figure(figsize=(10, 6))
sns.barplot(x='cohort', y='incident_rate', hue='cohort', data=cohort_incident_rate, palette='Set2')
plt.xticks(rotation=45)
plt.title('Tasa de incidentes por cohorte (%)')
plt.xlabel('Cohorte (Mes y Año)')
plt.ylabel('Tasa de incidentes (%)')
plt.show()

In [ ]:
# 14. Porcentaje de incidentes por cohorte y tipo de estado
print("\n\033[1m\033[95mPARTE 14: Porcentaje de incidentes por cohorte y tipo de estado\033[0m")

# 14.1 Contar el número de incidentes por cohorte y tipo de estado
incident_counts2 = incident_df.groupby(['cohort', 'status']).size().reset_index(name='incidents_count')

# 14.2 Obtener el total de incidentes por cohorte para calcular el porcentaje
total_incidents_per_cohort = incident_counts2.groupby('cohort')['incidents_count'].sum().reset_index(name='total_incidents')

# 14.3 Combinar las tablas para calcular el porcentaje
incident_counts2 = incident_counts2.merge(total_incidents_per_cohort, on='cohort')
incident_counts2['percentage'] = (incident_counts2['incidents_count'] / incident_counts2['total_incidents']) * 100

# 14.4 Visualización de los incidentes como porcentajes por cohorte y tipo de estado
plt.figure(figsize=(12, 8))
bar_plot = sns.barplot(data=incident_counts2, x='cohort', y='percentage', hue='status', palette='Set2')
plt.xticks(rotation=45)
plt.title('Porcentaje de incidentes por cohorte y tipo de estado')
plt.xlabel('Cohorte (Mes y Año)')
plt.ylabel('Porcentaje de incidentes (%)')
plt.legend(title='Tipo de incidente')
for p in bar_plot.patches:
    bar_plot.annotate(f'{p.get_height():.1f}%',
                      (p.get_x() + p.get_width() / 2., p.get_height()),
                      ha='center', va='bottom',
                      fontsize=10, color='black',
                      xytext=(0, 5),  # Desplazamiento vertical
                      textcoords='offset points')
plt.tight_layout()
plt.show()

In [ ]:
# 15. Porcentaje de incidentes por cohorte y tipo de estado apilado
print("\n\033[1m\033[95mPARTE 15: Porcentaje de incidentes por cohorte y tipo de estado apilado\033[0m")
# Visualización de los incidentes como porcentajes por cohorte y tipo de estado (gráfico apilado)
plt.figure(figsize=(14, 8))
# 15.1 Usamos pivot_table para reestructurar el DataFrame
incident_pivot = incident_counts2.pivot_table(index='cohort', columns='status', values='percentage', fill_value=0)
# 15.2 Graficar como un gráfico de barras apiladas
incident_pivot.plot(kind='bar', stacked=True, ax=plt.gca(), colormap='Set2')
plt.xticks(rotation=45)
plt.title('Porcentaje de incidentes por cohorte y tipo de estado (Gráfico Apilado)')
plt.xlabel('Cohorte (Mes y Año)')
plt.ylabel('Porcentaje de incidentes (%)')
plt.legend(title='Tipo de incidente')
plt.tight_layout()
plt.show()

In [ ]:
# 16 Desglose de incidencias por tipo y cantidad
print("\n\033[1m\033[95mPARTE 16: Desglose de incidencias por tipo y cantidad\033[0m")

# 16.1 Crear una tabla pivote para obtener la cantidad de incidencias por cohorte y tipo de incidencia
incident_pivot = incident_counts2.pivot_table(index='cohort', columns='status', values='incidents_count', fill_value=0)

# 16.2 Agrupar por cohorte y obtener la cantidad total de incidentes
cohort_incidents = incident_counts2.groupby('cohort').agg(
    total_incidents=('incidents_count', 'sum')
).reset_index()

# 16.3 Crear una figura y un eje
fig, ax1 = plt.subplots(figsize=(14, 8))

# 16.4 Graficar las barras apiladas para la cantidad de incidencias por tipo de incidente usando Set2
palette = sns.color_palette("Set2")
incident_pivot.plot(kind='bar', stacked=True, ax=ax1, color=palette)

# Convertir 'cohort' a formato datetime si no lo está ya
cohort_incidents['cohort'] = pd.to_datetime(cohort_incidents['cohort'], format='%Y-%m')

# 16.5 Establecer el título y etiquetas del eje y para el gráfico de barras
ax1.set_xlabel('Cohorte (Mes y Año)', fontsize=12)
ax1.set_ylabel('Cantidad de Incidencias por Tipo', fontsize=12)
plt.xticks(rotation=45)

# Formatear las etiquetas del eje X para mostrar solo año y mes
ax1.set_xticklabels(cohort_incidents['cohort'].dt.strftime('%Y-%m'))

# Añadir los porcentajes dentro de las barras
for i in range(len(incident_pivot)):
    total = cohort_incidents['total_incidents'].iloc[i]
    bottom = 0
    for j, val in enumerate(incident_pivot.iloc[i]):
        if val > 0:
            percentage = (val / total) * 100
            ax1.text(i + 0.3, bottom + val / 2, f'{percentage:.1f}%', ha='left', va='center', color='black', fontsize=10)
            bottom += val

# 16.6 Crear un segundo eje y para la cantidad total de incidencias
ax2 = ax1.twinx()

# 16.7 Graficar una línea para la cantidad total de incidencias por cohorte
sns.lineplot(x='cohort', y='total_incidents', data=cohort_incidents, ax=ax2, color='purple', marker='o')

# 16.8 Establecer etiquetas para el segundo eje y (cantidad total de incidencias)
ax2.set_ylabel('Cantidad Total de Incidencias', fontsize=12, color='purple')
ax2.tick_params('y', colors='purple')

# Añadir los valores totales encima de la línea
y_values = []
for i, total in enumerate(cohort_incidents['total_incidents']):
    ax2.text(i, total + 5, f'{int(total)}', ha='center', color='purple', fontsize=12)
    y_values.append(total)  # Almacenar los valores para la línea

# 16.9 Graficar la línea que conecta los totales
ax2.plot(cohort_incidents.index, y_values, color='purple', linestyle='--', linewidth=1.5)

# 16.9.1 Asegurar que ambos ejes compartan la misma escala
ax2.set_ylim(ax1.get_ylim())

# Invertir el orden de la leyenda
handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles[::-1], labels[::-1], title='Estado')

# 16.9.2 Visualizar gráfico
plt.title('Cantidad Total de Incidencias (Línea) y Porcentaje por Tipo de Incidencia (Barras Apiladas) por Cohorte', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# 17 Relación entre la Frecuencia de Usanza y las Ganancias de la Empresa
print("\n\033[1m\033[95mPARTE 17: Relación entre Frecuencia de Usanza y Ganancias de la Empresa (Agrupado)\033[0m")

# 17.1 Filtrar usuarios que han usado el servicio más de una vez
usuarios_multiples = df_cash['user_id'].value_counts()
usuarios_multiples = usuarios_multiples[usuarios_multiples > 1].index

# 17.2 Crear un DataFrame filtrado con usuarios que han usado el servicio más de una vez
df_cash_multiples = df_cash[df_cash['user_id'].isin(usuarios_multiples)]

# 17.3 Calcular la frecuencia de uso por usuario como el número total de solicitudes
frecuencia_por_usuario = df_cash_multiples.groupby('user_id')['id'].count().reset_index(name='frecuencia')

# 17.4 Unir las frecuencias al DataFrame de comisiones para obtener las ganancias por usuario
df_fees_filtered = df_fees[df_fees['status'] == 'accepted']
df_combinados = df_fees_filtered.merge(df_cash[['id', 'user_id']], left_on='cash_request_id', right_on='id', how='left')
df_combinados = df_combinados.merge(frecuencia_por_usuario, on='user_id', how='left')

# 17.5 Calcular las ganancias totales por usuario
ganancias_por_usuario = df_combinados.groupby('user_id')['total_amount'].sum().reset_index(name='ganancias')

# 17.6 Combinar las frecuencias y las ganancias en un solo DataFrame
df_frecuencia_ganancias = frecuencia_por_usuario.merge(ganancias_por_usuario, on='user_id')

# 17.7 Agrupar las ganancias por cada frecuencia única (sumando las ganancias de todos los usuarios con esa frecuencia)
df_frecuencia_ganancias_agrupadas = df_frecuencia_ganancias.groupby('frecuencia')['ganancias'].sum().reset_index()

# 17.8 Visualizar la relación mediante un gráfico de dispersión (scatter plot) con línea de tendencia
plt.figure(figsize=(10, 6))
scatter_plot = sns.scatterplot(x='frecuencia', y='ganancias', data=df_frecuencia_ganancias_agrupadas, palette='Set2', s=100)

# Añadir una línea de tendencia
sns.regplot(
    x='frecuencia', 
    y='ganancias', 
    data=df_frecuencia_ganancias_agrupadas, 
    scatter=False, 
    color='red', 
    line_kws={"linewidth":1.5, "linestyle":"--"}
)

# 17.9 Añadir etiquetas y título al gráfico
plt.title('Relación entre Frecuencia de Usanza y Ganancias de la Empresa (Agrupado)')
plt.xlabel('Frecuencia de Usanza (Solicitudes Totales)')
plt.ylabel('Ganancias Totales (en Unidades Monetarias)')
plt.grid(True)

# 17.10 Mostrar el gráfico
plt.show()


In [ ]:
# 17 Relación entre la Frecuencia de Usanza y las Ganancias de la Empresa (Agrupado)
print("\n\033[1m\033[95mPARTE 17: Relación entre Frecuencia de Usanza y Ganancias de la Empresa (Agrupado)\033[0m")

# 17.1 Filtrar usuarios que han usado el servicio más de una vez
usuarios_multiples = df_cash['user_id'].value_counts()
usuarios_multiples = usuarios_multiples[usuarios_multiples > 1].index

# 17.2 Crear un DataFrame filtrado con usuarios que han usado el servicio más de una vez
df_cash_multiples = df_cash[df_cash['user_id'].isin(usuarios_multiples)]

# 17.3 Calcular la frecuencia de uso por usuario como el número total de solicitudes
frecuencia_por_usuario = df_cash_multiples.groupby('user_id')['id'].count().reset_index(name='frecuencia')

# 17.4 Unir las frecuencias al DataFrame de comisiones para obtener las ganancias por usuario
df_fees_filtered = df_fees[df_fees['status'] == 'accepted']
df_combinados = df_fees_filtered.merge(df_cash[['id', 'user_id']], left_on='cash_request_id', right_on='id', how='left')
df_combinados = df_combinados.merge(frecuencia_por_usuario, on='user_id', how='left')

# 17.5 Calcular las ganancias totales por usuario
ganancias_por_usuario = df_combinados.groupby('user_id')['total_amount'].sum().reset_index(name='ganancias')

# 17.6 Combinar las frecuencias y las ganancias en un solo DataFrame
df_frecuencia_ganancias = frecuencia_por_usuario.merge(ganancias_por_usuario, on='user_id')

# 17.7 Agrupar las ganancias por cada frecuencia única (sumando las ganancias de todos los usuarios con esa frecuencia)
df_frecuencia_ganancias_agrupadas = df_frecuencia_ganancias.groupby('frecuencia')['ganancias'].sum().reset_index()

# 17.8 Filtrar para eliminar ganancias negativas
df_frecuencia_ganancias_agrupadas = df_frecuencia_ganancias_agrupadas[df_frecuencia_ganancias_agrupadas['ganancias'] >= 0]

# 17.9 Visualizar la relación mediante un gráfico de dispersión (scatter plot) con línea de tendencia logarítmica
plt.figure(figsize=(10, 6))
scatter_plot = sns.scatterplot(x='frecuencia', y='ganancias', data=df_frecuencia_ganancias_agrupadas, palette='Set2', s=100)

# Ajustar una línea de tendencia logarítmica
sns.regplot(
    x='frecuencia', 
    y='ganancias', 
    data=df_frecuencia_ganancias_agrupadas, 
    scatter=False, 
    color='red', 
    logx=True,  # Usar el logaritmo para el eje X
    line_kws={"linewidth": 1.5, "linestyle": "--"}
)

# 17.10 Añadir etiquetas y título al gráfico
plt.title('Relación entre Frecuencia de Usanza y Ganancias de la Empresa (Agrupado)')
plt.xlabel('Frecuencia de Usanza (Solicitudes Totales)')
plt.ylabel('Ganancias Totales (en Unidades Monetarias)')
plt.grid(True)

# 17.11 Mostrar el gráfico
plt.show()


In [ ]:
# 17 Relación entre la Frecuencia de Usanza y las Ganancias de la Empresa (Agrupado)
print("\n\033[1m\033[95mPARTE 17: Relación entre Frecuencia de Usanza y Ganancias de la Empresa (Agrupado)\033[0m")

# 17.1 Filtrar usuarios que han usado el servicio más de una vez
usuarios_multiples = df_cash['user_id'].value_counts()
usuarios_multiples = usuarios_multiples[usuarios_multiples > 1].index

# 17.2 Crear un DataFrame filtrado con usuarios que han usado el servicio más de una vez
df_cash_multiples = df_cash[df_cash['user_id'].isin(usuarios_multiples)]

# 17.3 Calcular la frecuencia de uso por usuario como el número total de solicitudes
frecuencia_por_usuario = df_cash_multiples.groupby('user_id')['id'].count().reset_index(name='frecuencia')

# 17.4 Unir las frecuencias al DataFrame de comisiones para obtener las ganancias por usuario
df_fees_filtered = df_fees[df_fees['status'] == 'accepted']
df_combinados = df_fees_filtered.merge(df_cash[['id', 'user_id']], left_on='cash_request_id', right_on='id', how='left')
df_combinados = df_combinados.merge(frecuencia_por_usuario, on='user_id', how='left')

# 17.5 Calcular las ganancias totales por usuario
ganancias_por_usuario = df_combinados.groupby('user_id')['total_amount'].sum().reset_index(name='ganancias')

# 17.6 Combinar las frecuencias y las ganancias en un solo DataFrame
df_frecuencia_ganancias = frecuencia_por_usuario.merge(ganancias_por_usuario, on='user_id')

# 17.7 Agrupar las ganancias por cada frecuencia única (sumando las ganancias de todos los usuarios con esa frecuencia)
df_frecuencia_ganancias_agrupadas = df_frecuencia_ganancias.groupby('frecuencia')['ganancias'].sum().reset_index()

# 17.8 Filtrar para eliminar ganancias negativas
df_frecuencia_ganancias_agrupadas = df_frecuencia_ganancias_agrupadas[df_frecuencia_ganancias_agrupadas['ganancias'] >= 0]

# 17.9 Visualizar la relación mediante un gráfico de dispersión (scatter plot) con línea de tendencia ajustada
plt.figure(figsize=(10, 6))
scatter_plot = sns.scatterplot(x='frecuencia', y='ganancias', data=df_frecuencia_ganancias_agrupadas, palette='Set2', s=100)

# Ajustar una línea de tendencia polinómica de orden 3
sns.regplot(
    x='frecuencia', 
    y='ganancias', 
    data=df_frecuencia_ganancias_agrupadas, 
    scatter=False, 
    color='red', 
    order=3,  # Ajuste cúbico
    line_kws={"linewidth": 1.5, "linestyle": "--"}
)

# Configurar el eje X para que vaya de 2 en 2
plt.xticks(range(0, df_frecuencia_ganancias_agrupadas['frecuencia'].max() + 2, 2))

# 17.10 Añadir etiquetas y título al gráfico
plt.title('Relación entre Frecuencia de Usanza y Ganancias de la Empresa (Agrupado)')
plt.xlabel('Frecuencia de Usanza (Solicitudes Totales)')
plt.ylabel('Ganancias Totales (en Unidades Monetarias)')
plt.grid(True)

# 17.11 Mostrar el gráfico
plt.show()


In [ ]:
# 18 Relación entre el porcentaje de incidentes y ganancias
print("\n\033[1m\033[95mPARTE 18: Relación entre porcentaje de incidentes y ganancias\033[0m")

# 18.1 Contar el número total de incidentes por cohorte
incident_counts_total = incident_df.groupby('cohort')['status'].count().reset_index(name='incident_count')

# 18.2 Calcular el total de solicitudes por cohorte para obtener el porcentaje de incidencias
total_requests_per_cohort = df_cash.groupby('cohort')['id'].count().reset_index(name='total_requests')

# 18.3 Calcular el porcentaje de incidentes por cohorte
incident_counts_total = incident_counts_total.merge(total_requests_per_cohort, on='cohort')
incident_counts_total['incident_percentage'] = (incident_counts_total['incident_count'] / incident_counts_total['total_requests']) * 100

# 18.4 Calcular las ganancias totales por cohorte (filtrando solo las solicitudes aceptadas)
cohort_revenues = df_aprobados.groupby('cohort')['total_amount'].sum().reset_index()

# 18.5 Unir los datos de incidentes (como porcentaje) y ganancias en un solo DataFrame
incidents_vs_revenue = incident_counts_total.merge(cohort_revenues, on='cohort')

# 18.6 Mostrar el DataFrame combinado
display(incidents_vs_revenue)

# 18.7 Crear un gráfico de dispersión relacionando el porcentaje de incidentes con las ganancias
plt.figure(figsize=(10, 6))
scatter_plot = sns.scatterplot(
    x='incident_percentage', 
    y='total_amount', 
    data=incidents_vs_revenue, 
    hue='cohort', 
    palette='Set2', 
    s=100
)

# Añadir una línea de tendencia
sns.regplot(
    x='incident_percentage', 
    y='total_amount', 
    data=incidents_vs_revenue, 
    scatter=False, 
    color='red', 
    line_kws={"linewidth":1.5, "linestyle":"--"}
)

# 18.8 Añadir etiquetas y título al gráfico
plt.title('Relación entre Porcentaje de Incidentes y Ganancias por Cohorte')
plt.xlabel('Porcentaje de Incidentes')
plt.ylabel('Ganancias Totales')
plt.legend(title='Cohorte (Mes y Año)', bbox_to_anchor=(1.05, 1), loc='upper left')

# 18.9 Añadir anotaciones para cada punto con los valores exactos de porcentaje de incidentes y ganancias
for i in range(incidents_vs_revenue.shape[0]):
    plt.text(
        x=incidents_vs_revenue['incident_percentage'].iloc[i], 
        y=incidents_vs_revenue['total_amount'].iloc[i], 
        s=f"({round(incidents_vs_revenue['incident_percentage'].iloc[i], 2)}%, {int(incidents_vs_revenue['total_amount'].iloc[i])})",
        fontdict=dict(color='black', size=8),
        ha='left', va='bottom'
    )

# 18.10 Mostrar el gráfico
plt.show()



In [ ]:
# PARTE 19: Tasa de Retención de Clientes
print("\n\033[1m\033[95mPARTE 19: Tasa de Retención de Clientes\033[0m")

# 19.1 Calcular el número de meses en los que cada cliente ha estado activo
df_cash['month_year'] = df_cash['date'].dt.to_period('M')  # Añadir columna con mes y año
retencion_por_usuario = df_cash.groupby('user_id')['month_year'].nunique().reset_index(name='meses_activo')

# 19.2 Calcular la tasa de retención (clientes que han estado activos más de un mes / total de clientes)
clientes_totales = retencion_por_usuario.shape[0]
clientes_retenidos = retencion_por_usuario[retencion_por_usuario['meses_activo'] > 1].shape[0]
tasa_retencion = (clientes_retenidos / clientes_totales) * 100

# 19.3 Crear un gráfico de barras para visualizar la tasa de retención
plt.figure(figsize=(8, 5))
plt.bar(['Clientes Retenidos', 'Clientes No Retenidos'], [clientes_retenidos, clientes_totales - clientes_retenidos], color=['green', 'red'])
plt.title('Tasa de Retención de Clientes')
plt.ylabel('Número de Clientes')
plt.xlabel('Estado del Cliente')
plt.show()

# Mostrar la tasa de retención en porcentaje
print(f"Tasa de Retención: {tasa_retencion:.2f}%")



In [ ]:
# PARTE 20: Valor del Tiempo de Vida del Cliente (CLV)
print("\n\033[1m\033[95mPARTE 20: Valor del Tiempo de Vida del Cliente (CLV)\033[0m")

# 20.1 Calcular el ingreso promedio por usuario
ingreso_por_usuario = df_cash.groupby('user_id')['total_amount'].sum().reset_index(name='ingreso_total')
ingreso_medio_por_usuario = ingreso_por_usuario['ingreso_total'].mean()

# 20.2 Calcular la retención promedio en meses por usuario
retencion_promedio_meses = retencion_por_usuario['meses_activo'].mean()

# 20.3 Calcular el CLV (ingreso medio * retención promedio)
clv = ingreso_medio_por_usuario * retencion_promedio_meses

# 20.4 Visualizar el CLV mediante un gráfico
plt.figure(figsize=(8, 5))
plt.bar(['Ingreso Medio por Usuario', 'Retención Promedio (meses)', 'CLV'], 
        [ingreso_medio_por_usuario, retencion_promedio_meses, clv], 
        color=['blue', 'orange', 'purple'])
plt.title('Valor del Tiempo de Vida del Cliente (CLV)')
plt.ylabel('Valor')
plt.xlabel('Componentes del CLV')
plt.show()

# Mostrar el CLV calculado
print(f"Valor del Tiempo de Vida del Cliente (CLV): {clv:.2f} unidades monetarias")


In [ ]:
# PARTE 21: Tasa de Conversión de Solicitudes a Ganancias
print("\n\033[1m\033[95mPARTE 21: Tasa de Conversión de Solicitudes a Ganancias\033[0m")

# 21.1 Calcular el número total de solicitudes y las solicitudes aceptadas
total_solicitudes = df_cash.shape[0]
solicitudes_aceptadas = df_fees_filtered['cash_request_id'].nunique()

# 21.2 Calcular la tasa de conversión
tasa_conversion = (solicitudes_aceptadas / total_solicitudes) * 100

# 21.3 Crear un gráfico de pastel para visualizar la tasa de conversión
plt.figure(figsize=(8, 5))
plt.pie([solicitudes_aceptadas, total_solicitudes - solicitudes_aceptadas], 
        labels=['Solicitudes Aceptadas', 'Solicitudes No Aceptadas'],
        autopct='%1.1f%%', startangle=90, colors=['green', 'gray'])
plt.title('Tasa de Conversión de Solicitudes a Ganancias')
plt.axis('equal')  # Para hacer el gráfico circular
plt.show()

# Mostrar la tasa de conversión en porcentaje
print(f"Tasa de Conversión de Solicitudes a Ganancias: {tasa_conversion:.2f}%")


FIN